# Training and testiong a ANN for the Titanic dataset

## Importing required libraries

In [289]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

## Defining constants

In [290]:
#Collumns that are not participate in the learning

class TicketFeatures:
    TICKET = 'Ticket'
    TICKET_PARSED = 'TicketParsed'
    TICKET_NUMBER = 'TicketNumber'
    TICKET_PREFIX = 'TicketPrefix'

class CabinFeatures:
    CABIN = 'Cabin'
    CABIN_PARSED = 'CabinParsed'
    CABIN_NUMBER = 'CabinNumber'
    DECK_CODE = 'DeckCode'

target_name = 'Survived'
    
features = ['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch','Embarked', 'TicketNumber', \
           'TicketPrefix', 'CabinNumber', 'DeckCode']

## Loading the Titanic dataset

In [291]:
titanic_df = pd.read_csv('../datasets/titanic/train.csv')
titanic_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Data description

Feature description:

 1. Survival - Survival (0 = No; 1 = Yes). Not included in test.csv file.
 2. Pclass - Passenger Class (1 = 1st; 2 = 2nd; 3 = 3rd)
 3. Name - Name
 4. Sex - Sex
 5. Age - Age
 6. Sibsp - Number of Siblings/Spouses Aboard
 7. Parch - Number of Parents/Children Aboard
 8. Ticket - Ticket Number
 9. Fare - Passenger Fare
 10. Cabin - Cabin
 11. Embarked - Port of Embarkation (C = Cherbourg; Q = Queenstown; S = Southampton)

## Data Structures

In [292]:
class Cabin:
    def __init__(self, deck_code, number):
        self.__deck_code = deck_code
        self.__number = number

    def __str__(self):
        return f'Cabin no. {self.__number}, deck code is "{self.__deck_code}"'
        
    @property    
    def deck_code(self):
        return self.__deck_code
    
    @deck_code.setter
    def deck_code(self, value):
        self.__deck_code = value
        
    @property    
    def number(self):
        return self.__number
    
    @number.setter
    def number(self, value):
        self.__number = value
            
    def parse(cabin):
        try:
            assert not (cabin is np.nan) 
            
            cabin_number = cabin[1 :]
            deck_code = cabin[0]
            
            return Cabin(deck_code, cabin_number)
        except ValueError as e:
            print(f'Unable parse the "{cabin}" cabin. Details: {e}')

class Ticket:
    def __init__(self, prefix, number):
        self.__prefix = prefix
        self.__number = number

    def __str__(self):
        
        ticket = ''
        if(not self.__prefix is np.nan):
            ticket = self.__prefix
        
        return f'Ticket {ticket} {self.__number} no. {self.__number}'
        
    @property    
    def prefix(self):
        return self.__prefix
    
    @prefix.setter
    def prefix(self, value):
        self.__prefix = value
        
    @property    
    def number(self):
        return self.__number
    
    @number.setter
    def number(self, value):
        self.__number = value
            
    def parse(ticket):
        try:
            ticket_number = np.nan
            ticket_prefix = np.nan
            ticket_splited = ticket.split(" ")
    
            if (ticket == 'LINE'):
                ticket_prefix = ticket
            elif (len(ticket_splited) == 1):
                ticket_number = int(ticket_splited[0])
            elif (len(ticket_splited) == 2):
                ticket_number = int(ticket_splited[1])
                ticket_prefix = str(ticket_splited[0])
            elif (len(ticket_splited) == 3):
                ticket_number = int(ticket_splited[2])
                ticket_prefix = f'{ticket_splited[0]} {ticket_splited[1]}'
            else:
                raise Exception(f'Unsupported ticket format.')
                
            return Ticket(ticket_prefix, ticket_number)
        except ValueError as e:
            print(f'Unable parse the "{ticket}" ticket. Details: {e}')

## Data preparation

In [293]:
#Set PassengerId as an index
titanic_df.set_index('PassengerId', inplace=True)

### Ticket feature preparation

In [294]:
null_ticket_filter = titanic_df[TicketFeatures.TICKET].isnull()
null_tickets_number = len(titanic_df[TicketFeatures.TICKET].loc[null_ticket_filter])

print(f'NULL tickets number is: {null_tickets_number}')

NULL tickets number is: 0


In [295]:
titanic_df[TicketFeatures.TICKET_PARSED] = titanic_df[TicketFeatures.TICKET].apply(lambda t : Ticket.parse(t))
titanic_df[TicketFeatures.TICKET_PARSED].head()

PassengerId
1             Ticket A/5 21171 no. 21171
2              Ticket PC 17599 no. 17599
3    Ticket STON/O2. 3101282 no. 3101282
4              Ticket  113803 no. 113803
5              Ticket  373450 no. 373450
Name: TicketParsed, dtype: object

In [296]:
#Create Ticket_Number feature
titanic_df[TicketFeatures.TICKET_NUMBER] =  titanic_df[TicketFeatures.TICKET_PARSED].apply(lambda t : t.number)
titanic_df[TicketFeatures.TICKET_NUMBER].head()

PassengerId
1      21171.0
2      17599.0
3    3101282.0
4     113803.0
5     373450.0
Name: TicketNumber, dtype: float64

In [297]:
#Create Ticket_Prefix feature
titanic_df[TicketFeatures.TICKET_PREFIX] =  titanic_df[TicketFeatures.TICKET_PARSED].apply(lambda t : t.prefix)
titanic_df[TicketFeatures.TICKET_PREFIX].head()

PassengerId
1         A/5
2          PC
3    STON/O2.
4         NaN
5         NaN
Name: TicketPrefix, dtype: object

In [298]:
#Printing ticket feature statisitcs
filt = titanic_df[TicketFeatures.TICKET_PREFIX].notnull()
not_null_ticket_prefixes_number = len(titanic_df.loc[filt])

filt = titanic_df[TicketFeatures.TICKET_NUMBER].notnull()
not_null_ticket_numbers_number = len(titanic_df.loc[filt])

print(f'Not NULL ticket prefixes number is: {not_null_ticket_prefixes_number}')
print(f'NULL ticket prefixes number is: {len(titanic_df) - not_null_ticket_prefixes_number}')
print(f'Not NULL ticket numbers number is: {not_null_ticket_numbers_number}')
print(f'NULL ticket numbers number is: {len(titanic_df) - not_null_ticket_numbers_number}')

Not NULL ticket prefixes number is: 230
NULL ticket prefixes number is: 661
Not NULL ticket numbers number is: 887
NULL ticket numbers number is: 4


### Cabin feature preparation

In [299]:
null_cabin_filter = titanic_df[CabinFeatures.CABIN].isnull()
null_cabins_number = len(titanic_df[CabinFeatures.CABIN].loc[null_cabin_filter])

print(f'NULL cabins number is: {null_cabins_number}')

NULL cabins number is: 687


In [300]:
not_null_cabin_filter = titanic_df[CabinFeatures.CABIN].notnull()
titanic_df[CabinFeatures.CABIN_PARSED] = titanic_df.loc[not_null_cabin_filter, CabinFeatures.CABIN] \
    .apply(lambda c : Cabin.parse(c))

titanic_df[CabinFeatures.CABIN_PARSED].head() 

PassengerId
1                                NaN
2     Cabin no. 85, deck code is "C"
3                                NaN
4    Cabin no. 123, deck code is "C"
5                                NaN
Name: CabinParsed, dtype: object

In [301]:
null_parsed_cabin_filter = titanic_df[CabinFeatures.CABIN_PARSED].notnull()
titanic_df[CabinFeatures.CABIN_NUMBER] = titanic_df.loc[null_parsed_cabin_filter, CabinFeatures.CABIN_PARSED] \
    .apply(lambda c : c.number)

titanic_df[CabinFeatures.CABIN_NUMBER].head()

PassengerId
1    NaN
2     85
3    NaN
4    123
5    NaN
Name: CabinNumber, dtype: object

In [302]:
titanic_df[CabinFeatures.DECK_CODE] = titanic_df.loc[null_parsed_cabin_filter, CabinFeatures.CABIN_PARSED] \
    .apply(lambda c : c.deck_code)

titanic_df[CabinFeatures.DECK_CODE].head()

PassengerId
1    NaN
2      C
3    NaN
4      C
5    NaN
Name: DeckCode, dtype: object

## Train and test dataset split

In [304]:
x_train, x_test, y_train, y_test = train_test_split(
    titanic_df[features], 
    titanic_df[target_name], 
    test_size=0.3, 
    random_state=123)

x_train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Embarked,TicketNumber,TicketPrefix,CabinNumber,DeckCode
PassengerId,,,,,,,,,,,
417,1,2,female,34.0,1,1,S,28220.0,NaN,NaN,NaN
802,1,2,female,31.0,1,1,S,31921.0,C.A.,NaN,NaN
513,1,1,male,36.0,0,0,S,17473.0,PC,25,E
456,1,3,male,29.0,0,0,C,349240.0,NaN,NaN,NaN
758,0,2,male,18.0,0,0,S,29108.0,NaN,NaN,NaN
